In [1]:
import torch
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data

import pandas as pd
import numpy as np
import warnings
import time

warnings.filterwarnings("ignore")

# ===================Train Data======================= #
#
#  输入30个采样点，预测30个采样点。由于每个
#  采样点都包含3个坐标(x,y,z)，所以需要输入
#  维度为90，输出维度为90。
#
#  每段轨迹长度3秒，共90个采样点。最后30个
#  采样点不需要作为输入，前30个采样点不需要
#  作为预测的真实值输出。前60个采样点作为输
#  入
#  
#  N = landmarks.shape[0] - landmarks.shape[0]/9
#
#  range取值是轨迹的条数
# =====================================================

# FC-net architecture
D_in, H_in, H_out, D_out = 30, 15, 15, 30 
Z_dim, Learning_rate = 3, 1e-3
#D_in, H_in, H_out, D_out = 20, 15, 15, 10

landmarks_frame = pd.read_csv('Try.csv')
landmarks = landmarks_frame.as_matrix().astype('float')

# Normalization
#maxdata = np.array([1.5, 0.4, 1.2])
#mindata = np.array([0.8, -0.5, 0.6])
maxdata = np.max(landmarks, axis=0)
mindata = np.min(landmarks, axis=0)
landmarks = (landmarks - mindata)/(maxdata - mindata)
landmarks = landmarks.astype('float').reshape(-1, D_in)


N = landmarks.shape[0] * 8 / 9
N = int(N)
count = landmarks.shape[0] / 9
count = int(count)
#print('Landmarks shape: {}'.format(landmarks.shape))
x = torch.zeros(N, D_in)
y = torch.zeros(N, D_out)

for i in range(count):
    x[i*8:i*8+8] = Variable(torch.from_numpy(landmarks[i*9:i*9+8]), requires_grad = False)
    y[i*8:i*8+8] = Variable(torch.from_numpy(landmarks[i*9+1:i*9+9, :D_out]), requires_grad = False)
print(x.shape)

torch.manual_seed(1)    # reproducible
torch_dataset = Data.TensorDataset(x, y)
# 把 dataset 放入 DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=8,               # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)


# ===================Validation Data======================= #
landmarks_frame_val = pd.read_csv('Val.csv')
landmarks_val = landmarks_frame_val.as_matrix().astype('float')

landmarks_val = (landmarks_val - mindata)/(maxdata - mindata)
landmarks_val = landmarks_val.astype('float').reshape(-1, D_in)

#print('Landmarks shape: {}'.format(landmarks_val.shape))

N_val = landmarks_val.shape[0] * 8 / 9
N_val = int(N_val)
count_val = landmarks_val.shape[0] / 9
count_val = int(count_val)

x_val = torch.zeros(N_val, D_in)
y_val = torch.zeros(N_val, D_out)

for i in range(count_val):
    x_val[i*8:i*8+8] = Variable(torch.from_numpy(landmarks_val[i*9:i*9+8]), requires_grad = False)
    y_val[i*8:i*8+8] = Variable(torch.from_numpy(landmarks_val[i*9+1:i*9+9, :D_out]), requires_grad = False)

print(x_val.shape)

torch.Size([544, 30])
torch.Size([152, 30])


In [18]:
""" VAE模型

自定义初始化
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

或调用官方方法
>>> w = torch.empty(3, 5)
>>> torch.nn.init.xavier_normal_(w)
"""

class vae(torch.nn.Module):
    def __init__(self, z_dim, x_dim, y_dim, h_dim):
        super(vae, self).__init__()
        self.Wxh = Variable(torch.nn.init.xavier_normal_(torch.empty(h_dim, x_dim)), requires_grad=True)
        self.bxh = Variable(torch.zeros(h_dim), requires_grad=True)
     
        self.Whz_mu = Variable(torch.nn.init.xavier_normal_(torch.empty(z_dim, h_dim)), requires_grad=True)  # 需要更改
        self.bhz_mu = Variable(torch.zeros(z_dim), requires_grad=True)

        self.Whz_var = Variable(torch.nn.init.xavier_normal_(torch.empty(z_dim, h_dim)), requires_grad=True)  # 需要更改
        self.bhz_var = Variable(torch.zeros(z_dim), requires_grad=True)

        #self.Wzh_tem = Variable(torch.nn.init.xavier_normal_(torch.empty(h_dim, z_dim)), requires_grad=True)
        #self.bzh_tem = Variable(torch.zeros(h_dim), requires_grad=True)
        
        self.Wzx = Variable(torch.nn.init.xavier_normal_(torch.empty(y_dim, z_dim)), requires_grad=True)  # 需要更改
        self.bzx = Variable(torch.zeros(y_dim), requires_grad=True)
        
        self.params = [self.Whz_mu, self.bhz_mu, self.Whz_var, self.bhz_var, self.Wzx, self.bzx]
        self.params.append(self.Wxh)
        self.params.append(self.bxh)
        #self.params.append(self.Wzh_tem)
        #self.params.append(self.bzh_tem)

    def forward(self, x):
        # See the autograd section for explanation of what happens here.
        self.z_mu, self.z_var = self.Q(x)
        z = self.sample_z(self.z_mu, self.z_var)
        y_pred = self.P(z)
        y_pred = y_pred + x
        return y_pred
    
    # =============================== Q(z|X) ======================================
    def Q(self, x):
        h = F.relu(F.linear(x, self.Wxh, self.bxh))
        z_mu = F.linear(h, self.Whz_mu, self.bhz_mu)
        z_var = F.linear(h, self.Whz_var, self.bhz_var)
        return z_mu, z_var

    def sample_z(self, mu, log_var):
        eps = Variable(torch.randn(mu.shape[0], mu.shape[1]))
        return mu + torch.exp(log_var / 2) * eps

    # =============================== P(X|z) ======================================
    def P(self, z):
        #h = F.linear(z, self.Wzh_tem, self.bzh_tem)
        X = F.linear(z, self.Wzx, self.bzx)
        return X
    
# =============================== TRAINING ====================================  
model = vae(z_dim = Z_dim, x_dim = D_in, y_dim = D_out, h_dim = H_in)   
optimizer = torch.optim.Adam(model.params, lr=Learning_rate)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [ ]:
# Whole batch

start = time.time()
for t in range(20000):
    y_pred = model(x)
    
    recon_loss = F.mse_loss(y_pred, y, size_average=False)
    kl_loss = torch.mean(0.5 * torch.sum(torch.exp(model.z_var) + model.z_mu**2 - 1. - model.z_var, 1))
    loss = recon_loss + kl_loss
    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #scheduler.step(loss)
    
    # Print and plot every now and then
    if t % 1000 == 0:
        print('Iter-{}; Loss: {:.4}'.format(t, recon_loss.data[0]))
end = time.time()
print('VAE net cal_time is {}'.format(end-start))
y_pred_val = model(x_val)
recon_loss_val = F.mse_loss(y_pred_val, y_val, size_average=False)
print('VAE val_loss is {}'.format(recon_loss_val.item()))

Iter-0; Loss: 3.213e+03
Iter-1000; Loss: 82.02
Iter-2000; Loss: 54.14
Iter-3000; Loss: 33.33
Iter-4000; Loss: 19.03
Iter-5000; Loss: 16.65
Iter-6000; Loss: 15.04
Iter-7000; Loss: 13.5
Iter-8000; Loss: 13.22
Iter-9000; Loss: 12.91
Iter-10000; Loss: 12.5
Iter-11000; Loss: 12.28
Iter-12000; Loss: 12.04
Iter-13000; Loss: 12.17
Iter-14000; Loss: 12.26
Iter-15000; Loss: 11.87
Iter-16000; Loss: 11.69
Iter-17000; Loss: 11.66
